# Pre-bootcamp exercises: accessing data products via butler


**Application to Auxtel data**

- Last Update : 2024/03/24
- w_2024_10

**Description:** Demonstrate how to generate science performance diagnostic plots and metrics with the [analysis_tools](https://github.com/lsst/analysis_tools) package using a small test dataset from HSC, [rc2_subset](https://github.com/lsst/rc2_subset).

**Contact authors:** Keith Bechtol, Nate Lust

**adapted for focal-plane-bootcamp by Sylvie Dagoret

**Last verified to run:** 2023-05-18

**LSST Science Piplines version:** w_2024_10

**Container Size:** Medium (or larger)

**Location:** This notebook points to files on the S3DF cluster at the USDF. Update paths accordingly if you are running elsewhere.

**Skills:** 
- Load source and object tables using the Butler.
- Generate a science performance diagnostic plot and corresponding metric values interactively in a notebook and as part of a pipeline (simple pipeline executor). 
- Adjust the configuration used to produce these diagnostics. 
- Retrieve persisted plots and metrics with the Bulter. 
- Reconstitute input data products that were used to create plots and metrics for further investigation.


## Preliminaries

In [1]:
# Basic imports
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from pprint import pprint

### Getting set up at USDF

The USDF is hosted on the S3DF cluster at SLAC. This notebook has been verified to run on the S3DF cluster.

See USDF documentation at
* https://developer.lsst.io/usdf/lsst-login.html
* https://developer.lsst.io/usdf/onboarding.html

### Setting up the analysis_tools package

Check the version of the stack you are using

In [2]:
!eups list -s | grep lsst_distrib

lsst_distrib          g4213664e8e+399544b0f1 	current w_2024_10 setup


The `analysis_tools` package was added to `lsst_distrib` in August 2022, and accordingly, if you have set up the LSST Stack version `w_2022_32` or later, then you should be able to import `analysis_tools` directly in the notebook.

See the header for this notebook for the most recent verified Science Pipelines version for this tutorial.

In [3]:
import lsst.analysis.tools
print(lsst.analysis.tools.__file__)

/home/d/dagoret/repos/repos_w_2024_10/analysis_tools/python/lsst/analysis/tools/__init__.py


**Additional background information:** If you are doing development on the `analysis_tools` package and want to test in a notebook, follow the guidance [here](https://nb.lsst.io/science-pipelines/development-tutorial.html). Brief version below (for work on the RSP at USDF):

1. In the termal, clone the [analysis_tools](https://github.com/lsst/analysis_tools) repo and set up the package

```
source /opt/lsst/software/stack/loadLSST.bash
setup lsst_distrib

# Choose file location for your repo
cd ~/repos/
git clone https://github.com/lsst/analysis_tools.git
cd analysis_tools
setup -k -r .
scons
```

2. Add the following line to `~/notebooks/.user_setups`

```
setup -k -r ~/repos/analysis_tools
```

Your local version of `analysis_tools` should now be accessible in a notebook.

## Load data for testing

In [4]:
import lsst.daf.butler as dafButler

In [5]:
repo = "/sdf/group/rubin/repo/oga/"   
collections = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"
print(collections)
print(repo)

LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871
/sdf/group/rubin/repo/oga/


In [6]:
butler = dafButler.Butler(repo, collections=collections)
registry = butler.registry

Check what dataset types are present in the collection. Note that the cell below will show _only the dataset types that are present in the specified collection_, not all of the possible dataset types that are known to the butler registry.

In [7]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=collections).any(execute=False, exact=False):
        print(datasetType)

DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure)
DatasetType('bias', {instrument, detector}, ExposureF, isCalibration=True)
DatasetType('camera', {instrument}, Camera, isCalibration=True)
DatasetType('dark', {instrument, detector}, ExposureF, isCalibration=True)
DatasetType('defects', {instrument, detector}, Defects, isCalibration=True)
DatasetType('flat', {band, instrument, detector, physical_filter}, ExposureF, isCalibration=True)
DatasetType('gaia_dr2_20200414', {htm7}, SimpleCatalog)
DatasetType('skyMap', {skymap}, SkyMap)
DatasetType('atlas_refcat2_20220201', {htm7}, SimpleCatalog)
DatasetType('packages', {}, Packages)
DatasetType('isr_config', {}, Config)
DatasetType('isr_metadata', {band, instrument, detector, physical_filter, exposure}, TaskMetadata)
DatasetType('isr_log', {band, instrument, detector, physical_filter, exposure}, ButlerLogRecords)
DatasetType('icSrc_schema', {}, SourceCatalog)
DatasetType('src_schema', {}, SourceCatalog)
Data

### Object tables

The examples below use an object table

In [8]:
refs = sorted(registry.queryDatasets("objectTable_tract"))
print(len(refs))

13


In [9]:
N = len(refs)
for idx in range(N):
    print(idx,refs[idx].dataId)

0 {skymap: 'latiss_v1', tract: 382}
1 {skymap: 'latiss_v1', tract: 383}
2 {skymap: 'latiss_v1', tract: 452}
3 {skymap: 'latiss_v1', tract: 455}
4 {skymap: 'latiss_v1', tract: 3863}
5 {skymap: 'latiss_v1', tract: 3864}
6 {skymap: 'latiss_v1', tract: 5614}
7 {skymap: 'latiss_v1', tract: 5615}
8 {skymap: 'latiss_v1', tract: 5616}
9 {skymap: 'latiss_v1', tract: 5634}
10 {skymap: 'latiss_v1', tract: 5839}
11 {skymap: 'latiss_v1', tract: 10643}
12 {skymap: 'latiss_v1', tract: 10644}


In [10]:
objectTable = butler.get(refs[5])
objectTable

column,ebv,detect_isTractInner,deblend_incompleteData,deblend_peak_center_x,detect_isDeblendedModelSource,coord_ra,detect_isIsolated,merge_peak_sky,deblend_parentTooBig,xy_flag,...,r_kronFlux_flag,r_kronFlux_flag_bad_radius,r_kronFlux_flag_bad_shape,r_kronFlux_flag_bad_shape_no_psf,r_kronFlux_flag_edge,r_kronFlux_flag_no_fallback_radius,r_kronFlux_flag_no_minimum_radius,r_kronFlux_flag_small_radius,r_kronFlux_flag_used_minimum_radius,r_kronFlux_flag_used_psf_radius
objectId,,,,,,,,,,,,,,,,,,,,,
1533950662142853121,1.114723,True,True,0,False,122.585256,True,False,False,True,...,True,False,True,False,True,False,False,False,False,False
1533950662142853122,1.092476,True,True,0,False,122.607274,True,False,False,False,...,True,False,True,False,False,False,False,False,False,False
1533950662142853123,1.083403,True,True,0,False,122.616231,True,False,False,True,...,False,True,False,False,False,False,False,True,False,True
1533950662142853124,1.098964,True,True,0,False,122.602018,True,False,False,False,...,True,False,True,False,False,False,False,False,False,False
1533950662142853125,1.103384,True,True,0,False,122.597953,True,False,False,False,...,True,True,True,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534016632840519781,1.966506,True,False,0,False,122.334704,True,True,False,True,...,True,False,False,False,False,False,False,False,False,False
1534016632840519782,2.064860,True,False,0,False,122.276713,True,True,False,True,...,True,False,False,False,False,False,False,False,False,False
1534016632840519885,1.903345,True,False,30380,True,122.371900,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [11]:
objectTable.columns.values

array(['ebv', 'detect_isTractInner', 'deblend_incompleteData',
       'deblend_peak_center_x', 'detect_isDeblendedModelSource',
       'coord_ra', 'detect_isIsolated', 'merge_peak_sky',
       'deblend_parentTooBig', 'xy_flag', 'deblend_failed',
       'deblend_nChild', 'coord_raErr', 'shape_flag',
       'detect_isDeblendedSource', 'deblend_peak_center_y',
       'coord_decErr', 'parentObjectId', 'deblend_skipped',
       'deblend_isolatedParent', 'detect_isPatchInner', 'yErr',
       'footprintArea', 'deblend_masked', 'shape_yy', 'refBand',
       'deblend_iterations', 'detect_fromBlend', 'coord_ra_dec_Cov',
       'deblend_nPeaks', 'shape_xx', 'detect_isPrimary',
       'refSizeExtendedness', 'sky_object', 'xErr', 'x', 'y', 'refFwhm',
       'shape_xy', 'refExtendedness', 'coord_dec', 'deblend_logL',
       'deblend_tooManyPeaks', 'tract', 'patch', 'g_ra', 'g_dec',
       'g_decl', 'g_raErr', 'g_decErr', 'g_ra_dec_Cov', 'g_psfFlux',
       'g_psfFluxErr', 'g_free_psfFlux', 'g_free_p

### Source tables

In [12]:
refs = sorted(registry.queryDatasets("sourceTable_visit"))

In [13]:
for idx,ref in enumerate(refs): 
    print(idx,ref.dataId.full)
    if idx> 10:
        break

0 {instrument: 'LATISS', visit: 2023051100262, band: 'r', physical_filter: 'SDSSr_65mm~empty'}
1 {instrument: 'LATISS', visit: 2023051100263, band: 'g', physical_filter: 'SDSSg_65mm~empty'}
2 {instrument: 'LATISS', visit: 2023051100264, band: 'i', physical_filter: 'SDSSi_65mm~empty'}
3 {instrument: 'LATISS', visit: 2023051100268, band: 'r', physical_filter: 'SDSSr_65mm~empty'}
4 {instrument: 'LATISS', visit: 2023051100270, band: 'i', physical_filter: 'SDSSi_65mm~empty'}
5 {instrument: 'LATISS', visit: 2023051100271, band: 'i', physical_filter: 'SDSSi_65mm~empty'}
6 {instrument: 'LATISS', visit: 2023051100272, band: 'g', physical_filter: 'SDSSg_65mm~empty'}
7 {instrument: 'LATISS', visit: 2023051100273, band: 'r', physical_filter: 'SDSSr_65mm~empty'}
8 {instrument: 'LATISS', visit: 2023051100274, band: 'r', physical_filter: 'SDSSr_65mm~empty'}
9 {instrument: 'LATISS', visit: 2023051100275, band: 'g', physical_filter: 'SDSSg_65mm~empty'}
10 {instrument: 'LATISS', visit: 2023051100276, ba

/tmp/ipykernel_24332/776994179.py:2: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print(idx,ref.dataId.full)


In [14]:
sourceTable = butler.get(refs[-1])
sourceTable.head()

,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,dec,...,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter
sourceId,,,,,,,,,,,,,,,,,,,,,
180918223816884225,122.544317,-36.218142,43134265856,0,2724.000000,28.000000,NaN,NaN,122.544317,-36.218142,...,False,False,False,False,False,True,2024013000170,0,y,empty~SDSSy_65mm
180918223816884226,122.566490,-36.217903,43134265856,0,3397.000000,28.000000,NaN,NaN,122.566490,-36.217903,...,False,False,False,False,False,True,2024013000170,0,y,empty~SDSSy_65mm
180918223816884227,122.455522,-36.218441,43134265856,0,29.161705,51.154515,1.824410,0.507562,122.455522,-36.218441,...,False,False,False,False,False,True,2024013000170,0,y,empty~SDSSy_65mm
180918223816884228,122.562855,-36.217305,43134265856,0,3287.000000,52.000000,0.611193,2.713681,122.562855,-36.217305,...,False,False,False,False,False,True,2024013000170,0,y,empty~SDSSy_65mm
180918223816884229,122.530090,-36.216850,43134265856,0,2292.895016,82.262679,0.138312,0.283244,122.530090,-36.216850,...,False,False,False,False,False,True,2024013000170,0,y,empty~SDSSy_65mm


In [15]:
sourceTable.columns.values

array(['coord_ra', 'coord_dec', 'ccdVisitId', 'parentSourceId', 'x', 'y',
       'xErr', 'yErr', 'ra', 'dec', 'decl', 'raErr', 'decErr',
       'ra_dec_Cov', 'calibFlux', 'calibFluxErr', 'ap03Flux',
       'ap03FluxErr', 'ap03Flux_flag', 'ap06Flux', 'ap06FluxErr',
       'ap06Flux_flag', 'ap09Flux', 'ap09FluxErr', 'ap09Flux_flag',
       'ap12Flux', 'ap12FluxErr', 'ap12Flux_flag', 'ap17Flux',
       'ap17FluxErr', 'ap17Flux_flag', 'ap25Flux', 'ap25FluxErr',
       'ap25Flux_flag', 'ap35Flux', 'ap35FluxErr', 'ap35Flux_flag',
       'ap50Flux', 'ap50FluxErr', 'ap50Flux_flag', 'ap70Flux',
       'ap70FluxErr', 'ap70Flux_flag', 'sky', 'skyErr', 'psfFlux',
       'psfFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF', 'iyyPSF', 'ixyPSF',
       'ixxDebiasedPSF', 'iyyDebiasedPSF', 'ixyDebiasedPSF',
       'gaussianFlux', 'gaussianFluxErr', 'extendedness',
       'sizeExtendedness', 'localPhotoCalib', 'localPhotoCalib_flag',
       'localPhotoCalibErr', 'localWcs_flag', 'localWcs_CDMatrix_2_1',
       '

## Run `analysis_tools` interactively

Run an `AnalysisTool` interactively in a notebook by passing in-memory data inputs to create metrics and diagnostic plots.

In this example, we compute PSF model size residuals relative to the observed PSF size.

In [16]:
from lsst.analysis.tools.atools import ShapeSizeFractionalDiff
from lsst.analysis.tools.interfaces._task import _StandinPlotInfo
from lsst.analysis.tools.interfaces import NoPlot

In [17]:
atool = ShapeSizeFractionalDiff()
atool.produce.plot.addSummaryPlot = False

# Do not produce plot; only metric values
#atool.produce.plot = NoPlot() 

# Apply the updated configuration
atool.finalize()

Notice that the returned metric values match summary statistics displayed on the plot

In [ ]:
results = atool(objectTable, band='g', skymap=None, plotInfo=_StandinPlotInfo())
results

{'ScatterPlotWithTwoHists': <Figure size 1920x1440 with 4 Axes>,
 'g_highSNStars_median': Measurement('g_highSNStars_median', <Quantity -0.00113466 pix>, notes={'g_highSNStars_median.metric_tags': []}),
 'g_highSNStars_sigmaMad': Measurement('g_highSNStars_sigmaMad', <Quantity 0.00632779 pix>, notes={'g_highSNStars_sigmaMad.metric_tags': []}),
 'g_highSNStars_count': Measurement('g_highSNStars_count', <Quantity 39. ct>, notes={'g_highSNStars_count.metric_tags': []}),
 'g_lowSNStars_median': Measurement('g_lowSNStars_median', <Quantity -0.00164639 pix>, notes={'g_lowSNStars_median.metric_tags': []}),
 'g_lowSNStars_sigmaMad': Measurement('g_lowSNStars_sigmaMad', <Quantity 0.00694808 pix>, notes={'g_lowSNStars_sigmaMad.metric_tags': []}),
 'g_lowSNStars_count': Measurement('g_lowSNStars_count', <Quantity 190. ct>, notes={'g_lowSNStars_count.metric_tags': []})}

## ConfigurableActions are the atomic bits of `analysis_tools`

We introduce core concepts of the `analysis_tools` package, starting with the idea of [ConfigurableAction](https://pipelines.lsst.io/v/weekly/modules/lsst.pex.config/overview.html#specialized-config-subclasses)s, or actions for short.

### Terminology

Data Types (many of the [actions in analysis_tools](https://github.com/lsst/analysis_tools/tree/main/python/lsst/analysis/tools/actions) are grouped according to the resultant data type)
* `Scalar`: Something that is number like (int, float, numpy.float32 etc.)
* `Vector`: Something that is ndarray like
* `KeyedData`: Anything that is indexed by a string that can return a Vector, or Scalar

Analysis Structures
* `ConfigurableAction`: generic interface for function-like objects (actions) that have state which can be set during configuration
* `AnalysisAction`: A ConfigurableAction subclass that is specialized for actions that function in analysis contexts
* `AnalysisTool`: A top level "container" of multiple AnalysisActions which performs one type of analysis

We dive into the later two in more detail below

### Using AnalysisActions

- Configurable `AnalysisActions` are the atomic bits of `analysis_tools`. They can be combined together to make more complex actions, or used as part of an AnalysisTool
- We show some examples of using configurable actions like standalone functions to provide intution for how configurable actions work.
- Show examples with KeyedDataActions, VectorActions (including selectors), and ScalarActions
- Show examples of configuration

Let's use actions to compute the measured PSF size for a set of stars from an object catalog.

In [ ]:
from lsst.analysis.tools.actions.vector import CalcMomentSize
from lsst.analysis.tools.actions.vector import ConvertFluxToMag

In [ ]:
sizeCalculator =  CalcMomentSize()

In [ ]:
# Inspect the configuration of this object.
pprint(sizeCalculator.toDict())

In [ ]:
# Inspect the required input schema, notice that we will need to provide the band information
sizeCalculator.getInputSchema()

In [ ]:
size = sizeCalculator(objectTable, band='i')
print(size)

In [ ]:
magConverter = ConvertFluxToMag()

In [ ]:
pprint(magConverter.toDict())

In [ ]:
magConverter.getInputSchema()

In the next example, we show how to configure an action

In [ ]:
# Another example, this time to convert fluxes to magnitudes
magConverter = ConvertFluxToMag()
magConverter.vectorKey='{band}_psfFlux'
mag = magConverter(objectTable, band='i')

In [ ]:
# Notice that the lines above is equiavalent to the following
magConverterAlternate = ConvertFluxToMag()
magConverterAlternate.vectorKey='i_psfFlux'
magAlternate = magConverterAlternate(objectTable)

# Confirm the results match
assert np.allclose(mag, magAlternate, equal_nan=True)

It is also possible set configuration of an action during initialization

In [ ]:
magConverterAlternate = ConvertFluxToMag(vectorKey='{band}_psfFlux')

assert np.allclose(
    magConverter(objectTable, band='i'), 
    magConverterAlternate(objectTable, band='i'), 
    equal_nan=True
)

Now make a plot

In [ ]:
plt.figure()
plt.scatter(mag, size, s=1)
plt.xlim(10.5, 22.)
plt.ylim(0, 10)
plt.xlabel('mag')
plt.ylabel('size')

Let's remake that simple plot now selecting only the stars

In [ ]:
from lsst.analysis.tools.actions.vector import StarSelector

In [ ]:
StarSelector().toDict()

In [ ]:
StarSelector().getInputSchema()

In [ ]:
starSelection = StarSelector()(objectTable, band='i')

In [ ]:
plt.figure()
plt.scatter(mag[starSelection], size.values[starSelection], s=1)
plt.xlim(10.0, 22.)
plt.ylim(0, 10)
plt.xlabel('mag')
plt.ylabel('size')

We can chain together `AnalysisAction`s, as in the following example that produces an equivalent plot. The `analysis_tools` package frequently uses this approach of chaining together `AnalysisAction`s.

In [ ]:
from lsst.analysis.tools.actions.vector import DownselectVector
from lsst.analysis.tools.actions.keyedData import AddComputedVector

In [ ]:
DownselectVector().toDict()

In [ ]:
DownselectVector().getInputSchema()

In [ ]:
AddComputedVector().toDict()

In [ ]:
AddComputedVector().getInputSchema()

In [ ]:
from lsst.analysis.tools.actions.vector import DownselectVector
from lsst.analysis.tools.actions.keyedData import AddComputedVector

band = "i"

# Add a new key that points to a vector of "size" values
objectTableDemo = AddComputedVector(action=CalcMomentSize, keyName="size")(objectTable, band=band)

# Note type(objectTable) is now a python dictionary instead of a pandas table, but since both
# "quack" like KeyedData they can be used interchangably

# Add a new key that points to a vector of "mag" values
objectTableDemo = AddComputedVector(
    action=ConvertFluxToMag(
        vectorKey='{band}_psfFlux'
    ),
    keyName="mag"
)(objectTableDemo, band=band)

# Example of a more explicit syntax
downselector = DownselectVector()
downselector.vectorKey = "size"
downselector.selector = StarSelector
size = downselector(objectTableDemo, band=band)

# Example of a more compact syntax
mag = DownselectVector(vectorKey="mag", selector=StarSelector)(objectTableDemo, band=band)

plt.figure()
plt.scatter(mag, size, s=1)
plt.xlim(10., 22.)
plt.ylim(0, 10)
plt.xlabel('mag')
plt.ylabel('size')

### Actions as a generic interface for data
Actions are not restricted to tables or products loaded from the butler, `KeyedData` could also be things like dictionaries of numpy arrays.

In [ ]:
import lsst.analysis.tools.actions
dir(lsst.analysis.tools.actions)

In [ ]:
from lsst.analysis.tools.actions.scalar import StdevAction

# create some KeyedData
data = {"randomData": np.random.normal(0, 3, 10000)}

# initialize an action, setting it to use the key set above
action = StdevAction(vectorKey="randomData")

#plt.figure()
#plt.hist(data["randomData"], bins=100, density=True)

# Run the action and print the results
print(f"The standard deviation is {action(data)}")

### Create a new action

In the example below, we define a new VectorAction to multiply a vector by a scalar.

In [ ]:
from lsst.analysis.tools.interfaces import VectorAction, KeyedData, KeyedDataSchema, Vector
from lsst.pex.config import Field
#rom lsstinterfaces import KeyedData, KeyedDataSchema

class MultiplyByScalar(VectorAction):
    """Multiply vector by a scalar value"""

    vectorKey = Field[str](doc="Key of vector which should be loaded")
    factor = Field[float](doc="Multiplicative factor", default=1.)

    def getInputSchema(self) -> KeyedDataSchema:
        return ((self.vectorKey, Vector),)

    def __call__(self, data: KeyedData, **kwargs) -> Vector:
        return np.array(self.factor * data[self.vectorKey.format(**kwargs)])

In [ ]:
action = MultiplyByScalar(vectorKey="randomData", factor=2.)
results = action(data)
assert np.allclose(results, 2. * data['randomData'])

### Three conceptual steps in an `AnalysisTool`: prep, process, produce

As mentioned AnalysisTools can be thought of as executable containers of AnalysisActions. There are three different AnalysisActions, referred to as stages, named prep, process, and produce.
* Prep: Responsible for any initial selection and filtering of data
* Process: This is where any transformations and/or calculations are made
* Produce: Generates final plot and/or metric objects

The following examples will:
* Walk through the three stages of running an analysis tool in sequential lines of code, passing the output of one step as input to the next step
* Examine intermediate results

In [ ]:
prepResults = atool.prep(objectTable, band='i')
processResults = atool.process(prepResults, band='i')
produceResults = atool.produce(processResults, band='i', skymap=None, plotInfo=_StandinPlotInfo())

Inspect the intermediate results

In [ ]:
#prepResults

In [ ]:
#processResults

## Workflow examples

### Running analysis_tools as part of a pipeline

* **All examples in this notebook should use the simple pipeline executor** (here is how you do it in a notebook)
* We have a PipelineTask for each data product. A task can run multiple AnalysisTools that each produce a set of plots or set of metrics and are subclasses of AnalysisPipelineTask.
* Discuss an example yaml pipeline file (load the yaml)
* Provide the command to run the pipeline
* Show how to configure the pipeline, e.g., turning on or off different metrics and plots or changing other parameters

**WARNING:** If you are using your own processed instance of rc_subset, run the following cells to run analysis_tools as part of a pipeline from the notebook. If you are pointing to a shared sandbox instance of rc2_subset, skip the remaining cells in this section.

In [ ]:
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline

# set up an output collection with your username
analysisToolsCollection = "u/%s/analysisToolsExampleAuxtel"%os.environ['USER']
print(analysisToolsCollection)

# this can be skipped if you already have a read writable butler setup (above is read only)
#butlerRW = SimplePipelineExecutor.prep_butler(repo, inputs=collections, output=analysisToolsCollection)
butlerRW = butler
# load in the pipeline to run
pipeline = Pipeline.from_uri("$ANALYSIS_TOOLS_DIR/pipelines/coaddQualityCore.yaml")

# override a configuration within a certain AnalysisTool
configKey = "atools.shapeSizeFractionalDiff.prep.selectors.snSelector.threshold"
pipeline.addConfigOverride("analyzeObjectTableCore", configKey, 400)

# Run only the PSF size residual tool
pipeline.addConfigOverride("analyzeObjectTableCore", "atools", None)
pipeline.addConfigOverride("analyzeObjectTableCore", "atools.shapeSizeFractionalDiff", ShapeSizeFractionalDiff)

# bands = ['g', 'r', 'z', 'y']
bands = ['i']
pipeline.addConfigOverride("analyzeObjectTableCore", "bands", bands)
pipeline.addConfigOverride("catalogMatchTract", "bands", bands)
pipeline.addConfigOverride("refCatObjectTract", "bands", bands)

# restrict processing to the same dataId used above
whereString = "tract = 3864 AND skymap = 'latiss_v1'"

Display the pipeline that is about to be run

In [ ]:
print(pipeline)

# NOT WORKING FR0M THERE

In [ ]:
#assert False
# Prevent the executor from dumping plots into the notebook
backend_ =  mpl.get_backend() 
mpl.use("Agg")

executor = SimplePipelineExecutor.from_pipeline(pipeline, where=whereString, butler=butlerRW)
quanta = executor.run(True)

# Restore the ability for plots to be put into the notebook
mpl.use(backend_)

In [ ]:

# Refresh our read-only butler to see the changes made. (It's generally a
# good idea to work on read-only things)
butler.registry.refresh()

## Access persisted metrics

Specify the collection that holds the results of running analysis_tools. If you are pointing to your own instance of rc2_subset, you did this already above prior to running the pipeline and the cell below is not needed.

In [ ]:

analysisToolsCollection = "u/dagoret/analysisToolsExample"

Check what dataset types exist in the new collection.

In [ ]:
# See what datasets exist; there should now be objectTableCore_metrics
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections=analysisToolsCollection).any(execute=False, exact=False):
        print(datasetType)

Access the persisted metrics.

In [ ]:
# Get the metric that was written
print(analysisToolsCollection)
refs = sorted(butler.registry.queryDatasets("objectTableCore_metrics", collections=analysisToolsCollection))
pprint(refs)
dataId = refs[-1].dataId
print(dataId)
print(refs[-1].run)
objectTable_metrics = butler.get("objectTableCore_metrics", dataId=dataId, collections=analysisToolsCollection)
pprint(objectTable_metrics)

Also, we can see that a plot has been persisted, but there isn't a way to visualize the plot from the notebook. In the next section, we'll use the `reconstructor` to recreate the plot in the notebook.

In [ ]:
refs = sorted(butler.registry.queryDatasets(
    "objectTableCore_i_shapeSizeFractionalDiff_ScatterPlotWithTwoHists", 
    collections=analysisToolsCollection)
)
print(refs[0])

# The following line will throw an error
# plot = butler.get(
#     "objectTableCore_i_shapeSizeFractionalDiff_ScatterPlotWithTwoHists",
#     dataId=refs[0].dataId,
#     collections=analysisToolsCollection
# )

### Reconstruct the inputs to an `AnalysisTool`

Analysis(Tools/Actions) allow the exact state of `AnalysisTool`s to be saved into the Butler when a pipeline is run. This allows a user to 'reconstruct' things as they were when the tools were executed. This aids in debugging and deep diving into the data.

Below is an example of reconstructing one of the tasks that was run in the Pipeline above.

In [ ]:
from lsst.analysis.tools.tasks.reconstructor import reconstructAnalysisTools

# Read in just one task
label = "analyzeObjectTableCore"
taskState, inputData = reconstructAnalysisTools(butler, 
                                                collection=analysisToolsCollection, 
                                                label=label, 
                                                dataId=dataId, 
                                                callback=None
)

We have access to the exact configuration that was used to run the analysis tools.

In [ ]:
pprint(taskState.toDict())

We also have access to the input data that were used to produce the diagnostics, in this case, the object table.

In [ ]:
inputData["data"]

Quick check to verify that the object table data are indeed the same.

In [ ]:
assert np.allclose(inputData["data"]['coord_ra'], objectTable['coord_ra'])

We can now reproduce diagnostic metrics and plots interactively in the notebook

In [ ]:
# The following configuration won't be needed in the future
taskState.atools.shapeSizeFractionalDiff.produce.plot.addSummaryPlot = False

taskState.atools.shapeSizeFractionalDiff(
    inputData['data'],
    band='i',
    skymap=None,
    plotInfo=_StandinPlotInfo()
)

Next, we change one of the configuration parameters to see how the results change. 

In this example, we raise the signal-to-noise threshold to 200. Notice that the metric values and plot change with this updated object selection criteria.

In [ ]:
# change some configuration to see the differences
taskState.atools.shapeSizeFractionalDiff.prep.selectors.snSelector.threshold = 200
taskState.atools.shapeSizeFractionalDiff(
    inputData['data'],
    band='i',
    skymap=None,
    plotInfo=_StandinPlotInfo()
)

As before, we can also step through the calculation to check intermediate steps.

In [ ]:
prepResults = taskState.atools.shapeSizeFractionalDiff.prep(objectTable, band='i')
processResults = taskState.atools.shapeSizeFractionalDiff.process(prepResults, band='i')
produceResults = taskState.atools.shapeSizeFractionalDiff.produce(processResults, band='i', skymap=None, plotInfo=_StandinPlotInfo())

## Create a new analysis tool

Example `AnalysisTool`s can be found in the [atools](https://github.com/lsst/analysis_tools/tree/main/python/lsst/analysis/tools/atools) directory of the package.

Now let's create our own `AnalysisTool`

In [ ]:
from lsst.analysis.tools.interfaces import AnalysisTool
from lsst.analysis.tools.actions.scalar import MedianAction, CountAction
from lsst.analysis.tools.actions.vector import SnSelector


class DemoTool(AnalysisTool):
    #parameterizedBand: bool = False
    
    def setDefaults(self):
        super().setDefaults()
        
        # select on high signal to noise obejcts
        # add in a signal to noise selector
        self.prep.selectors.snSelector = SnSelector()
        
        # set what key the selector should use when deciding SNR
        self.prep.selectors.snSelector.fluxType = "psfFlux"
        
        # select what threshold value is desireable for the selector
        self.prep.selectors.snSelector.threshold = 10
        
        # the final name in the qualification is used as a key to insert
        # the calculation into KeyedData
        self.process.calculateActions.median = MedianAction()
        self.process.calculateActions.median.vectorKey = "psfFlux"
        self.process.calculateActions.count = CountAction()
        self.process.calculateActions.count.vectorKey = "psfFlux"
        
        # tell the metic what the units are for the quantity
        self.produce.metric.units = {"median": "Jy",
                                     "count": "count"}
        
        # Rename the quanity prior to producing the Metric
        # (useful for resuable workflows that set a name toward the end of computation)
        # self.produce.metric.newNames = {"medianValueName": "DemoMetric"}

Examine the configuration of our new tool

In [ ]:
DemoTool().toDict()

Make some synthetic data

In [ ]:
# Make some synthetic data
size = 500
flux = np.logspace(1., 4., size)
fluxErr = np.sqrt(flux)
flux += np.random.normal(0, np.sqrt(flux), size)
data = {"psfFlux": flux, "psfFluxErr": fluxErr}

plt.figure()
plt.xscale('log')
plt.yscale('log')
plt.scatter(data['psfFlux'], data['psfFluxErr'])
plt.xlabel('psfFlux')
plt.ylabel('psfFluxErr')

Run the new analysis tool.

In [ ]:
demoTool = DemoTool()
demoTool.populatePrepFromProcess()

# We can configure as needed
demoTool.prep.selectors.snSelector.threshold = 50

demoTool(data)

We can inspect intermediate stages of the analysis.

In [ ]:
# Example: how many of the data points pass the SNR threshold?
len(demoTool.prep(data)['psfFlux'])